# Summarize Texts
In this notebook we summarize the paper using abstractive summarization (BART) so that it can be fed into OpenAI gpt3.5 API.

Papers can be long, so it is easy to exceed the 1024 token limit of BART.

For this reason we approach ti summarization by splitting the paper in multiple parts before summarization, and then returning the complete document summarized.

In [101]:
from transformers import BartForConditionalGeneration, AutoTokenizer

model_ckpt = "sshleifer/distilbart-cnn-6-6"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BartForConditionalGeneration.from_pretrained(model_ckpt)

Papers to summarize

In [ ]:
%store -r similar_docs
print(similar_docs[1].payload['body'])

Tokenize the documents

In [103]:
import torch
from tqdm import tqdm
import math

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_ids = [tokenizer(doc.payload['body'], padding='max_length', return_tensors='pt').to(device) for doc in tqdm(similar_docs)]

100%|██████████| 6/6 [00:00<00:00, 26.43it/s]


Split

In [86]:
len(input_ids[0]['input_ids'][0])

3678

In [ ]:
max_size = 1024

documents_tokenized = []

for input_id in input_ids:
    n_splits = math.ceil(len(input_id[0])/max_size)
    token_splits = []
    for index in list(range(n_splits)):
        if(index != n_splits-1):
            print(str(index*max_size) + " - " + str((index+1)*max_size))
            token_splits.append({ "input_ids": torch.tensor(input_id['input_ids'][0][index*max_size:(index+1)*max_size]).unsqueeze(0), 
                                "attention_mask": torch.tensor(input_id['attention_mask'][0][index*max_size:(index+1)*max_size]).unsqueeze(0)})
        else:
            print(str(index*max_size) + " - " + str(len(input_id[0])%max_size + index*max_size))
            token_splits.append({ "input_ids": torch.tensor(input_id['input_ids'][0][index*max_size:len(input_id[0])%max_size + index*max_size]).unsqueeze(0), 
                            "attention_mask": torch.tensor(input_id['attention_mask'][0][index*max_size:len(input_id[0])%max_size + index*max_size]).unsqueeze(0)})
    
    documents_tokenized.append(token_splits)

In [106]:
summaries = []

for doc_tokenized in tqdm(documents_tokenized):
    doc_summary = []
    for index in tqdm(list(range(len(doc_tokenized)))):
        doc_summary.append(model.generate(input_ids=doc_tokenized[index]['input_ids'], 
                            attention_mask=doc_tokenized[index]['attention_mask'],
                            min_length=16, 
                            max_length=64))
    summaries.append(doc_summary)

100%|██████████| 6/6 [05:04<00:00, 50.74s/it]


In [107]:
text_summaries = []

for summary in summaries:
    text_summary = ""

    for split in summary:
        extracted_summary = tokenizer.decode(split[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    
        if ("." in extracted_summary):
            text_summary += (".".join(extracted_summary.split(".")[0:-1])) + "\n"
        else:
            text_summary += extracted_summary + "\n"
    
    text_summaries.append(text_summary)

In [108]:
%store text_summaries

Stored 'text_summaries' (list)
